In [14]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [4]:
# Load the list of medication names from an Excel file
medications = pd.read_excel('Data/drugs_data_big2.xlsx')

medications= medications["name"]

In [51]:
data = pd.read_csv("downloads/modified_dataset_vanilla.csv")
data = data.dropna(subset=['Texto'])
data

,ID_DOENTE,PROCESSO,COD_REFERENCIA,COD_PZ,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,PROVENIENCIA,COD_CTH_PRIOR,CTH_PRIOR,...,DATA_REALIZACAO,OBSERVACOES,Mês_entrada,Ano_entrada,trata data recusa,resume saída,mês_saida,ano_saida,Texto,clean_text
0,SSHCCB,LVMVDV,LQHSOQLH,GDCADAB,3151575,USF TORRE DA MARINHA,CS/USF,CTH,3.0,Normal,...,2015-06-12,NaN,11,2014,2015-05-21,12/06/2015,6,2015,"Utente de 75 anos, refere tremor desde há 5 an...",75 ano tremor desde 5 ano esquecimento dificul...
1,CSAS,LVMVBD,LOHLTRTS,FZFZSDB,3152471,USF SERVIR SAUDE-CS,CS/USF,CTH,NaN,unknown,...,2012-07-02,NaN,4,2012,2012-04-23,02/07/2012,7,2012,Avaliação neurologica para avaliação da toma d...,avaliação neurologico avaliação tomar anti-epi...
2,ADCFD,LVMMVJ,LQHROQHL,NaN,0,NaN,OUTRA,SAM,NaN,unknown,...,2015-01-13,NaN,9,2014,2014-09-26,13/01/2015,1,2015,cefaleias,cefaleia
3,SSZDBZ,LVMMLV,LPHQULSU,CSGGHEZ,3151672,USF AMORA SAUDAVEL,CS/USF,CTH,NaN,unknown,...,2013-12-03,NaN,10,2013,2013-10-25,03/12/2013,12,2013,"Utente do sexo feminino com 30 anos de idade, ...",sexo feminino 30 ano idade aparentemente saidá...
4,SEAHEG,LVMNDL,LPHHVHVL,NaN,0,NaN,OUTRA,unknown,NaN,unknown,...,NaN,Conforme e-mail de 30/11/2015,2,2013,2013-02-26,26/02/2013,2,2013,Sem relatorio clinico,relatorio clinico
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1777,ZAEDSC,MIINNJJJJJ,LRHSSOHP,NaN,0,NaN,OUTRA,SAM,NaN,unknown,...,2016-06-27,NaN,10,2015,2016-02-25,27/06/2016,6,2016,Estenose carotidea assintomática bilateral < 7...,estenose carotidea assintomático bilateral < 7...
1778,AZGAFH,MIINCIL,LRHLHOOH,NaN,0,NaN,OUTRA,SAM,NaN,unknown,...,NaN,NaN,2,2015,2015-02-23,23/02/2015,2,2015,61 anos operada por patologia benigna da mama...,61 ano operar patologia benigno mama direito a...
1779,FGAZCD,MIINCBE,LRHOVHOP,ZHZHASSH,3151672,USF AMORA SAUDAVEL,CS/USF,CTH,NaN,unknown,...,NaN,NaN,5,2015,2015-07-14,14/07/2015,7,2015,Utente de 69A com Perda de conhecimento a 6.5....,69a perda conhecimento 6.5.2015 elipotomia tce...
1780,FGAZCD,MIINCBE,LRHRTLPV,ZHCHBECG,3151672,USF AMORA SAUDAVEL,CS/USF,CTH,3.0,Normal,...,NaN,NaN,9,2015,2016-02-08,08/02/2016,2,2016,Utente de 70A com historia de lipotimia em Mai...,70a historia lipotimer maio 2015 . eeg-29.5. 2...


In [17]:
# Create a binary label for each row of text indicating whether it mentions a medication
data['label'] = data['Texto'].apply(lambda x: any(medication in x for medication in medications))

# Split the data into training and test sets
train_df, test_df = train_test_split(data, test_size=0.2)

# Create a pipeline to vectorize the text and train a Naive Bayes classifier
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', MultinomialNB())
])

# Train the model on the training data
pipeline.fit(train_df['Texto'], train_df['label'])

# Evaluate the model on the test data
accuracy = pipeline.score(test_df['Texto'], test_df['label'])
print(f'Test accuracy: {accuracy:.2f}')

Test accuracy: 0.86


In [52]:
from Levenshtein import distance

# Define a function to check if a word is a close match to any medication name
def is_medication(word, medications, max_distance=2):
    for medication in medications:
        if distance(word, medication) <= max_distance:
            return True
    return False

# Create a binary label for each row of text indicating whether it mentions a medication
data['label'] = data['Texto'].apply(lambda x: any(is_medication(word, medications) for word in x.split()))

In [28]:
data["label"].value_counts()

True     1345
False     428
Name: label, dtype: int64

In [54]:
data=data.sort_values('DATA_RETORNO')
data = data.dropna(subset=['Texto'])

# compute the number of rows to select
n = int(len(data) * 0.8)

# select the first 80% of rows using slicing
train = data.iloc[:n]
#select the last 20%
test = data.iloc[n:]


# Create a pipeline to vectorize the text and train a Naive Bayes classifier
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', MultinomialNB())
])

# Train the model on the training data
pipeline.fit(train['Texto'], train['label'])

# Evaluate the model on the test data
accuracy = pipeline.score(test['Texto'], test['label'])
print(f'Test accuracy: {accuracy:.2f}')

Test accuracy: 0.75


In [55]:
medications = pd.read_excel('Data/drugs_data_big2.xlsx')

# Create a binary label for each row of text indicating whether it mentions a level 1 medication
data['level_1'] = data['Texto'].apply(lambda x: any(is_medication(word, medications.loc[medications['level'] == 1, 'name']) for word in x.split()))

# Create a binary label for each row of text indicating whether it mentions a level 2 medication
data['level_2'] = data['Texto'].apply(lambda x: any(is_medication(word, medications.loc[medications['level'] == 2, 'name']) for word in x.split()))

# Create a binary label for each row of text indicating whether it mentions a level 3 medication
data['level_3'] = data['Texto'].apply(lambda x: any(is_medication(word, medications.loc[medications['level'] == 3, 'name']) for word in x.split()))

data=data.sort_values('DATA_RETORNO')
data = data.dropna(subset=['Texto'])

# compute the number of rows to select
n = int(len(data) * 0.8)

# select the first 80% of rows using slicing
train = data.iloc[:n]
#select the last 20%
test = data.iloc[n:]

# Create three pipelines to vectorize the text and train a classifier for each label
pipelines = {
    'level_1': Pipeline([
        ('vectorizer', CountVectorizer()),
        ('classifier', MultinomialNB())
    ]),
    'level_2': Pipeline([
        ('vectorizer', CountVectorizer()),
        ('classifier', MultinomialNB())
    ]),
    'level_3': Pipeline([
        ('vectorizer', CountVectorizer()),
        ('classifier', MultinomialNB())
    ])
}

# Train the models on the training data
for level, pipeline in pipelines.items():
    pipeline.fit(train['Texto'], train[level])

# Evaluate the models on the test data
for level, pipeline in pipelines.items():
    accuracy = pipeline.score(test['Texto'], test[level])
    print(f'Test accuracy ({level}): {accuracy:.2f}')

Test accuracy (level_1): 0.83
Test accuracy (level_2): 0.90
Test accuracy (level_3): 0.92


In [59]:

# Make predictions on the data using the trained models
test['level_1_pred'] = pipelines['level_1'].predict(test['Texto'])
test['level_2_pred'] = pipelines['level_2'].predict(test['Texto'])
test['level_3_pred'] = pipelines['level_3'].predict(test['Texto'])

test

/var/folders/g5/fkg03q9j52z1zkf7tm78r7pr0000gn/T/ipykernel_39892/2494388086.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['level_1_pred'] = pipelines['level_1'].predict(test['Texto'])
/var/folders/g5/fkg03q9j52z1zkf7tm78r7pr0000gn/T/ipykernel_39892/2494388086.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['level_2_pred'] = pipelines['level_2'].predict(test['Texto'])
/var/folders/g5/fkg03q9j52z1zkf7tm78r7pr0000gn/T/ipykernel_39892/2494388086.py:4: SettingWithCopyWarning: 
A value is tryi

,ID_DOENTE,PROCESSO,COD_REFERENCIA,COD_PZ,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,PROVENIENCIA,COD_CTH_PRIOR,CTH_PRIOR,...,ano_saida,Texto,clean_text,label,level_1,level_2,level_3,level_1_pred,level_2_pred,level_3_pred
1720,AZSSZD,MIIJBMB,LSHPHVRT,ZDDHSEZS,3151571,USF FFMAIS-CS SEIXAL,CS/USF,CTH,3.0,Normal,...,2016,"Utente sexo fem, 76 anos Multipatologia - ver ...",sexo fem 76 ano multipatologer ver lista probl...,False,True,True,True,False,True,False
975,SAEHEB,LLCDCVC,LSHPVLTR,NaN,0,NaN,OUTRA,SAM,NaN,unknown,...,2016,"avc alt cog doent dr of, jp",avc alt cog doent of jp,False,False,True,False,False,True,False
458,SBEDSZ,VLMLCDI,LSHPVOVU,ZDEBECFZ,3150572,USF MONTE DA CAPARICA,CS/USF,CTH,NaN,unknown,...,2016,"25 anos, refere ter sido seguido em neuropedia...",25 ano ter ser seguir neuropediatria jovem epi...,True,True,True,False,True,True,False
1746,ZBGSDS,MIILBME,LSHPOPUL,ZDDESGSB,3151671,USF QUINTA DA ROSINHA,CS/USF,CTH,NaN,unknown,...,2016,Utente de 64 anos que refere alterações de mem...,64 ano alteração memória relacionar patologia ...,False,False,True,False,False,True,False
932,FDBDBC,LLCNJMD,LSHPOLUO,ZDDEEBBC,3150572,USF MONTE DA CAPARICA,CS/USF,CTH,NaN,unknown,...,2016,Sintomatologia: ALTERAÇÕES DA MEMÓRIA CURTA DU...,sintomatologia alteração memória curto duração...,True,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1607,FFDSBH,MILLLCJ,LSHRRULH,NaN,0,NaN,OUTRA,SAM,NaN,unknown,...,2016,Disfagia orofaringea/dificuldade de deglutição...,disfagia orofaringea / dificuldade deglutição ...,False,False,True,False,False,True,False
1643,SASEHD,MILNLLN,LQHHSLSU,CGGGCHE,3150502,CHARNECA DA CAPARICA,CS/USF,CTH,NaN,unknown,...,2014,TEM SIDO SEGUIDA CONS C. PLASTICA POS INT PE D...,ser seguir cons c. plastico pos int pe dto ha ...,False,False,False,False,False,True,False
1691,AHGEFG,MILDJDV,LSHOLVQL,NaN,0,NaN,OUTRA,SAM,NaN,unknown,...,2016,Agradeçia nova marcação consulta Dra. Luísa Co...,agradeçia novo marcação consulto dra Luísa doe...,False,False,False,False,False,True,False
1694,AHFHED,MILDMBJ,LSHQHVRS,NaN,0,NaN,OUTRA,SAM,NaN,unknown,...,2016,"Doente de 59 anos de idade, sexo masculino, co...",doente 59 ano idade sexo masculino antecedente...,True,False,True,False,False,True,False


In [64]:
test["level_1_pred"].value_counts()

False    293
True      62
Name: level_1_pred, dtype: int64

In [61]:
test["level_2"].value_counts()

True     294
False     61
Name: level_2, dtype: int64

In [60]:
test["level_3"].value_counts()

False    314
True      41
Name: level_3, dtype: int64